In [ ]:
# imports
import numpy as np
import pandas as pd
import json

In [ ]:
country_map = {
    "nl": "Netherlands",
    "fr": "France",
    "dk": "Denmark",
    "lu": "Luxembourg",
    "de": "Germany",
    "it": "Italy",
    "gb": "United Kingdom",
    "be": "Belgium",
    "ch": "Switzerland",
    "at": "Austria",
    "se": "Sweden",
    "mc": "Monaco",
    "no": "Norway",
    "yu": "Yugoslavia",
    "es": "Spain",
    "fi": "Finland",
    "pt": "Portugal",
    "ie": "Ireland",
    "mt": "Malta",
    "il": "Israel",
    "gr": "Greece",
    "tr": "Turkey",
    "ma": "Morocco",
    "cy": "Cyprus",
    "is": "Iceland",
    "hr": "Croatia",
    "ba": "Bosnia and Herzegovina",
    "si": "Slovenia",
    "pl": "Poland",
    "hu": "Hungary",
    "ru": "Russia",
    "sk": "Slovakia",
    "ro": "Romania",
    "ee": "Estonia",
    "lt": "Lithuania",
    "mk": "North Macedonia",
    "lv": "Latvia",
    "ua": "Ukraine",
    "cs": "Serbia and Montenegro",
    "al": "Albania",
    "ad": "Andorra",
    "by": "Belarus",
    "md": "Moldova",
    "bg": "Bulgaria",
    "am": "Armenia",
    "rs": "Serbia",
    "ge": "Georgia",
    "me": "Montenegro",
    "cz": "Czech Republic",
    "az": "Azerbaijan",
    "sm": "San Marino",
    "au": "Australia"
}

votes_df = pd.read_csv('votes.csv')
votes_df['to_country'] = votes_df['to_country'].map(country_map)
votes_df['from_country'] = votes_df['from_country'].map(country_map)

In [ ]:
sent_votes_df = votes_df[votes_df['round'] == 'final'].drop(columns=['round', 'from_country_id', 'to_country_id'])
sent_votes_df = sent_votes_df.groupby(['year', 'from_country', 'to_country']).sum()
sent_votes_df = sent_votes_df[sent_votes_df['total_points'] != 0]
sent_votes_df['tele_points'] = sent_votes_df['tele_points'].astype(int)
sent_votes_df['jury_points'] = sent_votes_df['jury_points'].astype(int)
sent_votes_df

In [ ]:
nested_json = {}

for year, year_df in sent_votes_df.groupby(level=0):
    year_df = year_df.droplevel(0)
    nested_json[year] = {}

    for from_country, from_df in year_df.groupby(level=0):
        from_df = from_df.droplevel(0)
        nested_json[year][from_country] = from_df.to_dict(orient="index")

In [ ]:
result_json = nested_json
with open("voting_map_data.json", "w") as f:
    json.dump(result_json, f, indent=4)